In [1]:
import umap
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.stats 
from scipy.ndimage import gaussian_filter1d
import glob
import sys
sys.path.append('./helpers')
import loadmat as lm
import helpers as helpers
import pandas as pd
from sklearn import linear_model
%matplotlib widget
from sklearn.decomposition import PCA

In [41]:
def calcFR(data,good_cells=None,t_edges = None):
    if good_cells is None:
        good_cells = data['sp']['cids'][data['sp']['cgs']==2]
    if t_edges is None:
        dt= 0.2;
        t_edges = np.arange(0,data['sp']['st'].max()+dt,dt)
    else:
        dt = np.mean(np.diff(t_edges))
    # count spikes in each time bin for each cell
    
    spikecount = np.full((len(good_cells),len(t_edges)-1),np.nan)
    
    for cell_idx in range(len(good_cells)):   
        spike_t = data['sp']['st'][data['sp']['clu']==good_cells[cell_idx]]
        spikecount[cell_idx,:] = np.histogram(spike_t,bins=t_edges)[0]

      
    spikecount = np.hstack((spikecount,np.zeros((spikecount.shape[0],1))))  
    spikerate = spikecount/dt
    spikes = np.transpose(spikerate)
    X = gaussian_filter1d(spikes, 2, axis=0)
    return spikes,X,t_edges
    

In [3]:
data = lm.loadmat(r'/Users/attialex/npJ5_0506_dark_1.mat')

In [42]:
spikes,X,t_edges = calcFR(data,t_edges = data['post'])

In [63]:
speed = helpers.calcSpeed(data['posx'])

In [54]:
plt.figure()
plt.plot(speed[speed>2])


/Users/attialex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
data_xcorr = lm.loadmat(r'/Users/attialex/npJ5_0506_dark_1_out.mat')

In [6]:
data_out=data_xcorr['data_out']

In [85]:
plt.figure()
plt.imshow(data_out['xcorrs'][data_out['pvals']<0.05,:])
plt.scatter(data_out['peak_loc_all'][data_out['pvals']<0.05]/5,np.arange(np.sum(data_out['pvals']<0.05)),s=4,c='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
# idx = np.logical_and(np.logical_and(data_out['pvals']<0.05,data_out['peak_loc_all']>60*5),data_out['peak_loc_all']<70*5)
idx = np.logical_and(np.logical_and(data_out['pvals']<0.05,data_out['peak_loc_all']>23*5),data_out['peak_loc_all']<41*5)
idx = data_out['pvals']<0.05

In [118]:
plt.close('all')

In [117]:
plt.figure()
plt.imshow(data_out['xcorrs'][idx,:],aspect='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
X=gaussian_filter1d(spikes[speed>2,:], 15, axis=0)
speed_ds = speed[speed>2]
speed_ds = speed_ds[::10]
X_z=scipy.stats.zscore(X[::10,idx],axis=1)
pca = PCA(n_components=6)
X_new=pca.fit_transform(X_z)
plt.figure()
plt.scatter(X_new[:,0],X_new[:,1],marker='.',s=4,c=speed_ds)
plt.plot(X_new[2100:2200,0],X_new[2100:2200,1],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
X_z.shape

(11688, 193)

In [90]:
X=gaussian_filter1d(spikes[speed>2,:], 15, axis=0)
speed_ds = speed[speed>2]
speed_ds = speed_ds[::10]
X_z=scipy.stats.zscore(X[::10,idx],axis=1)
pca = PCA(n_components=6)
X_new=pca.fit_transform(X_z)
plt.figure()
plt.scatter(X_new[:,0],X_new[:,1],marker='.',s=4,c=speed_ds)
plt.plot(X_new[2100:2200,0],X_new[2100:2200,1],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
dx = np.diff(X_new[:,0])
dy = np.diff(X_new[:,1])
dx = dx[::100]
dy = dy[::100]
X=X_new[0:-1:100,0]
Y=X_new[0:-1:100,1]
fig, ax = plt.subplots()
q = ax.quiver(X, Y, dx, dy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
plt.close('all')

In [121]:
# reducer = umap.UMAP(n_components=3,metric='cosine',n_neighbors=500,init='spectral',min_dist=0.8)
#reducer = umap.UMAP(n_components=3,metric='cosine',init='spectral',min_dist=0.8)
reducer = umap.UMAP(n_components=3,metric='cosine',min_dist = 1)
Xu = reducer.fit_transform(X_new)

In [122]:
plt.figure()
plt.scatter(Xu[:,0],Xu[:,1],marker='.',s=4,c=speed_ds)
plt.plot(Xu[2800:3000,0],Xu[2800:3000,1],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
plt.figure()
plt.scatter(Xu[:,0],Xu[:,1],marker='.',s=4,c=speed_ds)
plt.plot(Xu[2800:3000,0],Xu[2800:3000,1],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
dx = np.diff(Xu[:,0])
dy = np.diff(Xu[:,1])
dx = dx[::10]
dy = dy[::10]
X=Xu[0:-1:10,0]
Y=Xu[0:-1:10,1]
fig, ax = plt.subplots()
q = ax.quiver(X, Y, dx, dy)

/Users/attialex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xu[:,0],Xu[:,1],Xu[:,2],marker='.',s=4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
import matplotlib;matplotlib.__version__

'3.1.1'